In [47]:
### start_imports ###
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
### end_imports ###
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### start_miscellaneous ###
from datetime import date,datetime,timedelta
hoje =  pd.to_datetime(date.today()) 

final_do_mes = pd.to_datetime(datetime(hoje.year,hoje.month+1,1) - timedelta(seconds=1))

biu_download_cols  = ['vmonitoramentoleilao', 'vmonitoramentoug', 'vmonitoramentousina', 'vrapeelacesso', 'vrapeelcontratorecurso', 'vrapeelcronograma', 'vrapeelempreendimento', 'vrapeellicenciamento', 'vrapeeloperacaoug']
biu_path = os.path.join(get_standard_folder_path("Documents"),"Previsor/BIU")

def ultimo_rapeel(df):
    # Função que retorna o último rapeel. Não usar groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax() 
    # pois não se quer o último rapeel de cada UG e sim as UGs dos últimos rapeel. 
    df_ultimo_envio = df.loc[df.groupby("IdeUsinaOutorga")['DthEnvio'].idxmax()][['IdeUsinaOutorga','DthEnvio']]
    return pd.merge(df_ultimo_envio,df,on=["IdeUsinaOutorga","DthEnvio"],how="left")


list_carac_usina = ['Usina_Sem_Rapeel','CondicaoAmbiental','homologar_marcos','revisar_IO','Usina_Selecionada','Paralisada','Sem_Previsao','criterio_novo_pmo','prev_OC_SFG_passado','em_teste','sem_monitoramento','manual','UG_sem_rapeel']
list_carac_ug = ['FASE','flagOPTeste30dias','UG_sem_rapeel','criterio_novo_pmo']
list_casos = ['caso_I','caso_II_a','caso_II_b','caso_III']
list_ambiental = ['DatValidadeLI','DatValidadeLP','DatValidadeLO','ValidadeAmbiental','CondicaoAmbiental']
list_contratos = ['NomSitContratoCCD', 'NomSitContratoCUST', 'NomSituacaoContratoCUSD', 'NomSitContratoCCT']
### end_miscellaneous ###

In [48]:
#### start_function ### function_name = generate_BIU
### start_function_vars ###
### start_funcion_defined_vars ###
### end_funcion_defined_vars ###
biu_download_path = os.path.join(biu_path,"Download")
previsao_file = os.path.join(biu_path,"Estagio_I/Previsao_OC.gzip")
### end_function_vars ###

In [49]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [50]:
cols = ["IdeUsinaOutorga",'NomUsina','CodCegFormatado','DatMonitoramento',"DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado",'DscJustificativaPrevisao']
# Carrega informações do monitoramento
df_usina = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentousina.gzip"))[cols].rename(
    columns= {
        'DatPrevisaoIniciobra'  : 'prev_IO_SFG'
    }
)

#Transforma coluna em booleano
df_usina.IdcUsinaMonitorada = df_usina.IdcUsinaMonitorada.map({"Sim":"True","Não":False}).astype(bool)

# Remove usinas que não são monitoradas
df_usina = df_usina[df_usina.IdcUsinaMonitorada].reset_index(drop=True)

In [51]:
# Excluir ? Não está sendo usado
monitoramentoleilao  = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
# Mantém apenas primeira data de suprimento 
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [52]:
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO_rapeel',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(biu_download_path,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]

# Faz merge left entre as informações do monitoramento e do rapeel
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [53]:
# Cria coluna para identificar usinas que não enviaram rapeel
df_usina["Usina_Sem_Rapeel"] = True
df_usina.loc[df_usina.IdeUsinaOutorga.isin(rapeel_cronograma.IdeUsinaOutorga),'Usina_Sem_Rapeel'] = False

In [54]:
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(biu_download_path,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [55]:
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(biu_download_path,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [56]:
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(biu_download_path,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [57]:
df_usina["classe"] = df_usina.CodCegFormatado.str.slice(4,6)
df_usina["CC_real"] = pd.to_datetime(pd.NA)
df_usina.loc[df_usina.classe == "PH","CC_real"] = df_usina.DatRealizacaoIV
df_usina.loc[df_usina.classe == "CV","CC_real"] = df_usina.DatRealizacaoV
df_usina["ME_real"] = df_usina.DatRealizacaoX
df_usina.loc[df_usina.classe == "CV","ME_real"] = df_usina.DatRealizacaoVI

In [58]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
cols = ['IdeUsinaOutorga'] + list(dict_validades.keys()) + ['ValidadeAmbiental']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental,IdeUsinaOutorga (case),DatValidadeLO (case),DatValidadeLI (case),DatValidadeLP (case),ValidadeAmbiental (case),Count
2,659,2026-02-22,2028-06-28,2000-03-29,2026-02-22,True,True,True,True,True,771
0,324,NaT,2026-03-09,2002-12-11,2026-03-09,True,False,True,True,True,741
25,28616,NaT,NaT,2013-05-21,2013-05-21,True,False,False,True,True,642
1,601,NaT,NaT,NaT,NaT,True,False,False,False,False,313
2302,54659,NaT,2026-10-22,NaT,2026-10-22,True,False,True,False,True,7
7,1997,2024-01-22,NaT,2011-11-17,2024-01-22,True,True,False,True,True,3


In [59]:
df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'

cols = ['IdeUsinaOutorga'] +  list(dict_validades) + ['DthEnvio','ValidadeAmbiental','CondicaoAmbiental','NomUsina']

display(show_cases(df_usina,cols))

df_usina['condicao_conexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'condicao_conexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'condicao_conexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'condicao_conexao'] = "Sem Conexão"

cols = ['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','condicao_conexao','NomUsina']

display(show_cases(df_usina,cols))



df_usina['condicao_uso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'condicao_uso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'condicao_uso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'condicao_uso'] = "Sem Uso"


cols = ['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','condicao_uso','NomUsina']
display(show_cases(df_usina,cols))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

cols = ['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA','NomUsina']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,DthEnvio,ValidadeAmbiental,CondicaoAmbiental,NomUsina,IdeUsinaOutorga (case),DatValidadeLO (case),DatValidadeLI (case),DatValidadeLP (case),DthEnvio (case),ValidadeAmbiental (case),CondicaoAmbiental (case),NomUsina (case),Count
2,659,2026-02-22,2028-06-28,2000-03-29,2023-01-06 09:37:52.703,2026-02-22,LO,Capivari,True,True,True,True,True,True,LO,True,771
0,324,NaT,2026-03-09,2002-12-11,2023-01-02 08:34:07.347,2026-03-09,LI,Bugres,True,False,True,True,True,True,LI,True,741
25,28616,NaT,NaT,2013-05-21,2023-01-02 16:42:33.840,2013-05-21,LP,Sumidouro,True,False,False,True,True,True,LP,True,642
3,1195,NaT,NaT,NaT,NaT,NaT,Não Informado,Itumirim,True,False,False,False,False,False,Não Informado,True,244
1,601,NaT,NaT,NaT,2023-01-03 19:22:43.140,NaT,Sem LP,Caldeirões,True,False,False,False,True,False,Sem LP,True,69
2302,54659,NaT,2026-10-22,NaT,2023-01-05 17:50:40.930,2026-10-22,LI,Padre Bernardo III,True,False,True,False,True,True,LI,True,7
7,1997,2024-01-22,NaT,2011-11-17,2023-01-02 14:04:26.367,2024-01-22,LO,Passo de Ajuricaba,True,True,False,True,True,True,LO,True,3


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,condicao_conexao,NomUsina,IdeUsinaOutorga (case),NomSitContratoCCD (case),NomSitContratoCCT (case),DthEnvio (case),condicao_conexao (case),NomUsina (case),Count
12,27835,Não se Aplica,Não Assinado,2023-01-04 20:32:15.297,Sem Conexão,Bazan,True,Não se Aplica,Não Assinado,True,Sem Conexão,True,1084
0,324,Não se Aplica,Válido,2023-01-02 08:34:07.347,OK,Bugres,True,Não se Aplica,Válido,True,OK,True,764
3,1195,NaN,NaN,NaT,Não informado,Itumirim,True,NaN,NaN,False,Não informado,True,244
7,1997,Não se Aplica,Não se Aplica,2023-01-02 14:04:26.367,Verificar,Passo de Ajuricaba,True,Não se Aplica,Não se Aplica,True,Verificar,True,151
6,1928,Válido,Não se Aplica,2023-01-05 19:34:10.547,OK,Pacífico Mascarenhas,True,Válido,Não se Aplica,True,OK,True,98
1,601,Não Assinado,Não se Aplica,2023-01-03 19:22:43.140,Sem Conexão,Caldeirões,True,Não Assinado,Não se Aplica,True,Sem Conexão,True,81
25,28616,Não Assinado,Não Assinado,2023-01-02 16:42:33.840,Sem Conexão,Sumidouro,True,Não Assinado,Não Assinado,True,Sem Conexão,True,51
2,659,Válido,Válido,2023-01-06 09:37:52.703,OK,Capivari,True,Válido,Válido,True,OK,True,2
619,38369,Não se Aplica,Vencido,2023-01-02 17:59:56.687,Sem Conexão,Mauriti 6,True,Não se Aplica,Vencido,True,Sem Conexão,True,1
41,29505,Vencido,Não se Aplica,2023-01-05 12:39:59.160,Sem Conexão,Córrego Fundo,True,Vencido,Não se Aplica,True,Sem Conexão,True,1


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,condicao_uso,NomUsina,IdeUsinaOutorga (case),NomSituacaoContratoCUSD (case),NomSitContratoCUST (case),condicao_uso (case),NomUsina (case),Count
50,30105,Não se Aplica,Válido,OK,Nardini Aporé,True,Não se Aplica,Válido,OK,True,936
46,29716,Não se Aplica,Não Assinado,Sem Uso,Nova Trento,True,Não se Aplica,Não Assinado,Sem Uso,True,844
3,1195,NaN,NaN,Não informado,Itumirim,True,NaN,NaN,Não informado,True,244
0,324,Válido,Não se Aplica,OK,Bugres,True,Válido,Não se Aplica,OK,True,169
7,1997,Não se Aplica,Não se Aplica,Verificar,Passo de Ajuricaba,True,Não se Aplica,Não se Aplica,Verificar,True,114
1,601,Não Assinado,Não se Aplica,Sem Uso,Caldeirões,True,Não Assinado,Não se Aplica,Sem Uso,True,99
25,28616,Não Assinado,Não Assinado,Sem Uso,Sumidouro,True,Não Assinado,Não Assinado,Sem Uso,True,52
2,659,Válido,Válido,OK,Capivari,True,Válido,Válido,OK,True,11
264,34829,Não se Aplica,Vencido,Sem Uso,Ventos de São Vitor 5,True,Não se Aplica,Vencido,Sem Uso,True,3
18,28000,Vencido,Vencido,Sem Uso,Nhandu,True,Vencido,Vencido,Sem Uso,True,2


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA,NomUsina,IdeUsinaOutorga (case),DscComercializacaoEnergia (case),DatConclusaoACL (case),PPA (case),NomUsina (case),Count
0,324,Fora do ACR,NaT,Nenhum,Bugres,True,Fora do ACR,False,Nenhum,True,1964
34,29116,Fora do ACR,2022-03-04,ACL,Santo Ângelo,True,Fora do ACR,True,ACL,True,252
2,659,ACR,NaT,ACR,Capivari,True,ACR,False,ACR,True,220
44,29576,ACR,2013-04-03,Ambos,Bela Vista,True,ACR,True,Ambos,True,41


In [60]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

In [61]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.condicao_uso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.condicao_uso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.condicao_uso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.condicao_uso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


cols = ['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','condicao_uso','criterio_novo']
display(show_cases(df_usina,cols,max_cases=10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,condicao_uso,criterio_novo,IdeUsinaOutorga (case),IdcSituacaoObra (case),PPA (case),CondicaoAmbiental (case),condicao_uso (case),criterio_novo (case),Count
29,28791,Não Iniciada,Nenhum,LP,Sem Uso,5,True,Não Iniciada,Nenhum,LP,Sem Uso,5,342
93,32076,Não Iniciada,Nenhum,LO,Sem Uso,3,True,Não Iniciada,Nenhum,LO,Sem Uso,3,316
156,32996,Não Iniciada,Nenhum,LO,OK,2,True,Não Iniciada,Nenhum,LO,OK,2,272
99,32145,Não Iniciada,Nenhum,Não Informado,Não informado,7,True,Não Iniciada,Nenhum,Não Informado,Não informado,7,240
43,29526,Não Iniciada,Nenhum,LI,Sem Uso,3,True,Não Iniciada,Nenhum,LI,Sem Uso,3,184
49,30101,Não Iniciada,Nenhum,LP,OK,4,True,Não Iniciada,Nenhum,LP,OK,4,172
10,27165,Não Iniciada,Nenhum,LI,OK,2,True,Não Iniciada,Nenhum,LI,OK,2,169
67,30666,Em andamento,Nenhum,LI,OK,0,True,Em andamento,Nenhum,LI,OK,0,75
34,29116,Em andamento,ACL,LO,OK,0,True,Em andamento,ACL,LO,OK,0,74
109,32305,Em andamento,ACL,LI,OK,0,True,Em andamento,ACL,LI,OK,0,62


In [62]:
dict_marcos_homologar = {
    'DatInicioObraRealizado' : 'IO_real',
    'DatConcretagemRealizado' : 'CC_real',
    'DatMontagemRealizado' : 'ME_real',
    'DatEnchimentoRealizado' : 'Ench_Real',
    'DatSisTransmissaoRealizado' : 'IOTrans_Real',
    'DatCanteiroObraRealizado' : 'canteiroReal',
    'DatComissionamentoUGRealizado' : 'Comiss_Real',
    'DatConclusaoSisTransRealizado' : 'Conc_Trans_REAL',
    'DatDesvioRioRealizado' : 'DesvRio_real',
    'DatConclusaoTorresRealizado' : 'ME_Real_conc_eol'}
    
df_usina['homologar_marcos'] = False
df_usina['dsc_marcos_a_homologar'] = ''

# Homologação de marcos

for marco_monitoramento, marco_rapeel in dict_marcos_homologar.items():
    mask = (df_usina[marco_monitoramento].isna()) & (df_usina[marco_rapeel].notna())
    df_usina['homologar_marcos'] |= mask
    df_usina.loc[mask,'dsc_marcos_a_homologar'] += f'{marco_monitoramento}, '

df_usina.dsc_marcos_a_homologar = df_usina.dsc_marcos_a_homologar.str.slice(0,-2)

cols = ["IdeUsinaOutorga",'NomUsina'] + ['homologar_marcos','dsc_marcos_a_homologar']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,NomUsina,homologar_marcos,dsc_marcos_a_homologar,IdeUsinaOutorga (case),NomUsina (case),homologar_marcos (case),dsc_marcos_a_homologar (case),Count
0,324,Bugres,False,,True,True,False,True,2367
8,2586,Salto Cafesoca,True,DatSisTransmissaoRealizado,True,True,True,True,110


In [63]:
df_usina['revisar_IO'] =  np.select(
 [(df_usina.prev_IO_SFG < final_do_mes) |
 (df_usina.prev_IO_rapeel > df_usina.prev_IO_SFG)],
 [True],
 default = False)
cols = ['IdeUsinaOutorga', 'NomUsina','prev_IO_SFG','prev_IO_rapeel','revisar_IO']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,NomUsina,prev_IO_SFG,prev_IO_rapeel,revisar_IO,IdeUsinaOutorga (case),NomUsina (case),prev_IO_SFG (case),prev_IO_rapeel (case),revisar_IO (case),Count
10,27165,Emas Nova,2023-03-01,2023-03-01,False,True,True,True,True,False,1649
3,1195,Itumirim,NaT,NaT,False,True,True,False,False,False,387
58,30355,Codora,2023-11-23,NaT,False,True,True,True,False,False,255
0,324,Bugres,NaT,2025-02-01,False,True,True,False,True,False,121
119,32483,Mutamba IV,2023-02-16,2023-02-16,True,True,True,True,True,True,62
584,38134,Ventos de São Zacarias 09,2023-02-28,NaT,True,True,True,True,False,True,3


In [64]:
usinas_selecionadas = read_file("./usinas_selecionadas.txt").split(",")
usinas_selecionadas = [int(num) for num in usinas_selecionadas]
df_usina["Usina_Selecionada"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(usinas_selecionadas),'Usina_Selecionada'] = True

df_usina['Paralisada'] = False
df_usina.loc[df_usina.IdcObraParalisada=="Sim",'Paralisada'] = True

df_usina['Sem_Previsao'] = False
df_usina.loc[df_usina.IdcSemPrevisao=="Sim",'Sem_Previsao'] = True

cols = ['IdeUsinaOutorga', 'NomUsina','Usina_Selecionada','Paralisada','Sem_Previsao']
display(show_cases(df_usina,cols))

,IdeUsinaOutorga,NomUsina,Usina_Selecionada,Paralisada,Sem_Previsao,IdeUsinaOutorga (case),NomUsina (case),Usina_Selecionada (case),Paralisada (case),Sem_Previsao (case),Count
6,1928,Pacífico Mascarenhas,False,False,False,True,True,False,False,False,2297
0,324,Bugres,False,False,True,True,True,False,False,True,86
11,27243,Santa Cruz,True,False,False,True,True,True,False,False,47
15,27969,Itaocara I,True,False,True,True,True,True,False,True,27
2,659,Capivari,False,True,True,True,True,False,True,True,13
53,30150,Almirante Álvaro Alberto - Unidade III (Antiga...,True,True,True,True,True,True,True,True,6
597,38221,Galvani Serra do Salitre,False,True,False,True,True,False,True,False,1


# Informações de UGs

## Carrega banco de dados

In [65]:
cols = ["IdeUsinaOutorga",'NumUgUsina',"IdcMonitorada",'DatLiberOpComerRealizado','DatPrevisaoSFGComercial']
monitoramentoug = pd.read_parquet(os.path.join(biu_download_path,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})
# Não remover ugs não monitoradas nesse passo, pois serão usadas para encontrar ugs difentes entre 
# monitoramento e rapeel. São retiradas durante o merge
monitoramentoug.IdcMonitorada = monitoramentoug.IdcMonitorada.map({"Sim":True,"Não":False}).astype(bool)

display(show_cases(monitoramentoug,monitoramentoug.columns))

,IdeUsinaOutorga,NumOperacaoUg,IdcMonitorada,DatLiberOpComerRealizado,DatPrevisaoSFGComercial,IdeUsinaOutorga (case),NumOperacaoUg (case),IdcMonitorada (case),DatLiberOpComerRealizado (case),DatPrevisaoSFGComercial (case),Count
6,27165,2,True,NaT,2025-11-18,True,True,True,False,True,71359
0,742,1,False,1962-12-14,NaT,True,True,False,True,False,14808
2,1358,1,True,NaT,NaT,True,True,True,False,False,3293
1136,32921,1,False,NaT,2027-04-06,True,True,False,False,True,434
9,27422,3,False,NaT,NaT,True,True,False,False,False,420
61,29243,1,False,2020-03-12,2020-04-15,True,True,False,True,True,322


In [66]:
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(biu_download_path,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ultimo_rapeel(ug_rapeel)
display(show_cases(ug_rapeel,ug_rapeel.columns))

,IdeUsinaOutorga,DthEnvio,NumOperacaoUg,DatPrevistaComercial,IdeUsinaOutorga (case),DthEnvio (case),NumOperacaoUg (case),DatPrevistaComercial (case),Count
0,273,2022-10-09 21:04:17.743,4,2023-11-02,True,True,True,True,81877
6,659,2023-01-06 09:37:52.703,2,NaT,True,True,True,False,2562


In [67]:
cols_used_robot = ['DatInicioObraOutorgado','prev_IO_SFG','prev_IO_rapeel']
list_cols_used =   ['IdeUsinaOutorga','IdcObraParalisada','IdcSemPrevisao','criterio_novo','Usina_Sem_Rapeel',] + cols_used_robot
# UGs não monitoradas são retiradas aqui
df_ug = pd.merge(df_usina[list_cols_used],monitoramentoug[monitoramentoug.IdcMonitorada],on="IdeUsinaOutorga",how='left')
df_ug.IdcMonitorada = df_ug.IdcMonitorada.astype(bool)
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left') # poderia ser inner?
# Remove usinas que já entraram em operação comercial ou usinas sem UGs monitoradas
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna() &  df_ug.NumOperacaoUg.notna()].reset_index(drop=True)

# Remove usinas que não possuem nenhuma UG monitorada
df_usina = df_usina[df_usina.IdeUsinaOutorga.isin(df_ug.IdeUsinaOutorga)].reset_index(drop=True)

In [68]:
# Carrega informações do previsor

calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

cols = ["Dat_OC_obrigacao","prev_IO_SFG","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg','Previsao_OC':'calculo_previsor_OC'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)[['IdeUsinaOutorga','NumOperacaoUg','FASE','Ind_crono_norm','flagOPTeste30dias','calculo_previsor_OC','Dat_OC_obrigacao','MdaPotenciaUnitaria']]

df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")
display(show_cases(calculo_previsao,calculo_previsao.columns))

,IdeUsinaOutorga,NumOperacaoUg,FASE,Ind_crono_norm,flagOPTeste30dias,calculo_previsor_OC,Dat_OC_obrigacao,MdaPotenciaUnitaria,IdeUsinaOutorga (case),NumOperacaoUg (case),FASE (case),Ind_crono_norm (case),flagOPTeste30dias (case),calculo_previsor_OC (case),Dat_OC_obrigacao (case),MdaPotenciaUnitaria (case),Count
0,1358,1,OUT,4.650318e-03,0,2025-07-15 13:24:30.102,2016-02-01,3500.0,True,True,OUT,True,0,True,True,True,62618
15,30482,3,IO,8.458899e-01,0,2024-02-14 10:47:40.782,2025-01-01,16000.0,True,True,IO,True,0,True,True,True,5719
8,28818,2,ME,8.970399e-01,0,2023-05-25 20:53:40.592,2025-01-01,11151.0,True,True,ME,True,0,True,True,True,4664
83,31685,8,OT,5.922102e-07,0,2023-02-03 07:58:45.697,2019-03-01,3437.0,True,True,OT,True,0,True,True,True,827
10,29433,2,CC,7.106852e-01,0,2024-01-27 08:37:14.522,2023-07-31,4500.0,True,True,CC,True,0,True,True,True,533
22,27243,1,OT,4.121207e-04,1,2023-04-11 09:25:05.356,2014-12-01,150000.0,True,True,OT,True,1,True,True,True,291


In [69]:
monitoramentoug['existe_monitoramento'] = True     # Adiciona colunas para checagem
ug_rapeel['existe_rapeel'] = True                  # dos resultados

# Faz merge tipo outer (todas as UGs) com os dados do  
# monitoramento e rapeel

comp_ug = pd.merge(monitoramentoug[list_id_ug + ['existe_monitoramento',"IdcMonitorada"]], 
        ug_rapeel[list_id_ug + ['existe_rapeel']], how="outer")                            


# Inner join com usinas monitoradas
comp_ug = pd.merge(df_usina[df_usina.IdcUsinaMonitorada][['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga")



# Inner join com usinas que já enviaram rapeel
comp_ug = pd.merge(rapeel_cronograma[['IdeUsinaOutorga']],comp_ug,how="inner",on="IdeUsinaOutorga") 



# Das UGs que estão no monitoramento, remove-se UGs não monitoradas
comp_ug = comp_ug[comp_ug.IdcMonitorada != False]



# Preenche com o valor falso as usinas que não foram encontradas
comp_ug[['existe_monitoramento','existe_rapeel']] = comp_ug[['existe_monitoramento','existe_rapeel']].fillna(False) 



# Coluna com diferenças
comp_ug["Diff"] = ~(comp_ug.existe_monitoramento & comp_ug.existe_rapeel) 


df_UG_sem_rapeel = comp_ug[comp_ug.existe_monitoramento & comp_ug.Diff][['IdeUsinaOutorga']].drop_duplicates()
df_UG_sem_rapeel['UG_sem_rapeel'] = True


# Todas as UGs de uma usina que pelo menos uma UG não possua rapeel terão UG_sem_rapeel = True
df_ug = pd.merge(df_ug,df_UG_sem_rapeel,on='IdeUsinaOutorga',how='left')
df_ug.loc[df_ug.UG_sem_rapeel.isna(),'UG_sem_rapeel'] = False


# Adiona a coluna PrevisaoOC_rapeel_max, que é a máxima previsão OC dentre todas as UGs enviadas no rapeel, independente se estão
# no monitoramento ou não.
df_ug = df_ug.merge(ug_rapeel.loc[ug_rapeel[ug_rapeel.DatPrevistaComercial.notna()].groupby("IdeUsinaOutorga").DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial']].rename(columns={'DatPrevistaComercial':'PrevisaoOC_rapeel_max'}),
            how="left",on="IdeUsinaOutorga")


# Para esses casos, em vez de DatPrevistaComercial, será usado PrevisaoOC_rapeel_max
# Ver arquivo 
df_ug.loc[df_ug.UG_sem_rapeel | df_ug.DatPrevistaComercial.isna(),'DatPrevistaComercial'] = df_ug.PrevisaoOC_rapeel_max


##  Cria colunas com definições e regras

In [70]:
################################ criterio_novo_pmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterio_novo_pmo'] = np.select(list_condicoes,list_values)

################################ regranovapmo ################################
df_ug.loc[(df_ug.criterio_novo_pmo == 0.1),'regranovapmo'] = df_ug[['DatPrevistaComercial','calculo_previsor_OC']].max(axis=1)

df_ug.loc[(df_ug.criterio_novo_pmo == 0.2),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))


df_ug.loc[(df_ug.criterio_novo_pmo == 0.3) & (df_ug['calculo_previsor_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']


df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['calculo_previsor_OC'].notna()),'regranovapmo'] = df_ug['calculo_previsor_OC']


df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','calculo_previsor_OC','Dat_OC_obrigacao']].max(axis=1)

df_ug["Handicap"] = hoje + pd.Timedelta(5*365,'D')
df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','calculo_previsor_OC','Dat_OC_obrigacao','Handicap']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Handicap"] = hoje + pd.Timedelta(6*365,'D')

df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['calculo_previsor_OC','Dat_OC_obrigacao','Handicap']].max(axis=1)


df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA

del df_ug['Handicap']

In [71]:
serie_usinas_previsao_OC_passado = df_ug[df_ug.DatPrevisaoSFGComercial < final_do_mes].IdeUsinaOutorga.drop_duplicates()

serie_usinas_previsao_em_teste = df_ug[df_ug.criterio_novo_pmo.isin([0.2,0.3])].IdeUsinaOutorga.drop_duplicates()

serie_flagOPTeste30dias = df_ug[df_ug.flagOPTeste30dias == 1].IdeUsinaOutorga.drop_duplicates()
df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

serie_UG_sem_rapeel = df_ug[df_ug.UG_sem_rapeel].IdeUsinaOutorga.drop_duplicates()

In [72]:
df_usina["UG_sem_rapeel"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_UG_sem_rapeel),'UG_sem_rapeel'] = True

df_usina["flagOPTeste30dias"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_flagOPTeste30dias),'flagOPTeste30dias'] = True

df_usina["prev_OC_SFG_passado"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_OC_passado),'prev_OC_SFG_passado'] = True

df_usina["em_teste"] = False
df_usina.loc[df_usina.IdeUsinaOutorga.isin(serie_usinas_previsao_em_teste),'em_teste'] = True

df_usina["sem_monitoramento"] = False
df_usina.loc[df_usina.DatMonitoramento < (hoje - pd.Timedelta(4*30, unit="D")),'sem_monitoramento'] = True

In [73]:
df_usina["manual"] = False
df_usina.loc[(df_usina.Paralisada | df_usina.Sem_Previsao | 
    df_usina.Usina_Selecionada  | df_usina.em_teste | 
    (df_usina.DscJustificativaPrevisao == 'Situação das obras de conexão e linha de transmissão associada.')),
    'manual'] = True

In [74]:
# Casos de seleção

df_usina['caso_I'] = df_usina['caso_II_a'] = df_usina['caso_II_b'] = df_usina['caso_III'] = False
list_casos = ['caso_I','caso_II_a','caso_II_b','caso_III']

df_usina['caso_I'] = (~df_usina.manual) & (~df_usina.revisar_IO) & (~ df_usina.homologar_marcos) 

df_usina['caso_II_a'] = (~df_usina.manual) & (df_usina.revisar_IO) & (~ df_usina.homologar_marcos)

df_usina['caso_II_b'] = (~df_usina.manual)  & ((df_usina.homologar_marcos) | (df_usina.homologar_marcos & df_usina.revisar_IO))

df_usina['caso_III'] = (df_usina.manual & (df_usina.sem_monitoramento | df_usina.homologar_marcos | df_usina.revisar_IO | df_usina.prev_OC_SFG_passado)) | df_usina.em_teste

df_usina['selecionado_BIU'] = df_usina[list_casos].any(axis=1)

In [75]:
list_criterios = ['criterio_novo',"criterio_novo_pmo"]

df_usina_criterio = df_ug.loc[df_ug.groupby("IdeUsinaOutorga").criterio_novo_pmo.idxmax()][["IdeUsinaOutorga","FASE",'flagOPTeste30dias'] + list_criterios]

################################ dsc_justificativa_regra_nova ################################
list_condicoes = [
    (df_usina_criterio['criterio_novo_pmo'] == 9),
    (df_usina_criterio['criterio_novo_pmo'] == 8),
    (df_usina_criterio['criterio_novo_pmo'] == 0.2),
    (df_usina_criterio['criterio_novo_pmo'] == 0.1) | (df_usina_criterio['criterio_novo_pmo'] == 0.3),
    (df_usina_criterio['criterio_novo_pmo'] == 1),
    df_usina_criterio['criterio_novo_pmo']==2,
    df_usina_criterio['criterio_novo_pmo']==3,
    (df_usina_criterio.criterio_novo_pmo== 4),
    (df_usina_criterio.criterio_novo_pmo== 5),
    (df_usina_criterio.criterio_novo_pmo== 6),
    (df_usina_criterio.criterio_novo_pmo== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_usina_criterio['dsc_justificativa_regra_nova'] = np.select(list_condicoes,list_values)

################################ dsccriterionovo ################################
list_condicoes = [
    ((df_usina_criterio['criterio_novo'] == 9)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] != "OT")),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT") & (df_usina_criterio['flagOPTeste30dias'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 0) & (df_usina_criterio['FASE'] == "OT")),
    ((df_usina_criterio['criterio_novo'] == 1)),
    ((df_usina_criterio['criterio_novo'] == 2)),
    ((df_usina_criterio['criterio_novo'] == 3)),
    ((df_usina_criterio['criterio_novo'] == 4)),
    ((df_usina_criterio['criterio_novo'] == 5)),
    ((df_usina_criterio['criterio_novo'] == 7)),
    ((df_usina_criterio['criterio_novo'] == 6)),
    ((df_usina_criterio['criterio_novo'] == 8))
]

list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_usina_criterio['dsccriterionovo'] = np.select(list_condicoes,list_values)
list_justificativas = ['dsccriterionovo','dsc_justificativa_regra_nova']

In [76]:
df_usina = pd.merge(df_usina,
df_usina_criterio[['IdeUsinaOutorga','criterio_novo_pmo'] + list_justificativas],
on = "IdeUsinaOutorga", how='left')


list_casos = ['caso_I','caso_II_a','caso_II_b','caso_III','selecionado_BIU']
df_ug = pd.merge(df_usina[['IdeUsinaOutorga','dsc_justificativa_regra_nova','dsccriterionovo','DatMonitoramento','DscJustificativaPrevisao'] + list_casos],df_ug, on = "IdeUsinaOutorga",how='left')

In [77]:
# Renomeia colunas para exportação
dic = {
    'NumOperacaoUg':'NumUgUsina',
    'regranovapmo': 'PrevisaoOC_regra',
    'dsc_justificativa_regra_nova' : 'Justificativadaprevisao_new',
    'criterio_novo_pmo' : 'CriterioPrevisao',
    'dsccriterionovo':'DscCriterioPrevisao',
    'Previsao_OC':'CalculoPrevisorOC',
    'Dat_OC_obrigacao':'OC_Obrigacao',
    'DscJustificativaPrevisao':'DscJustificativaPrevisaoAtual',
    'calculo_previsor_OC':'CalculoPrevisorOC'
}   


df_ug = df_ug.rename(columns=dic)
df_usina = df_usina.rename(columns=dic)

In [78]:
""" ### include_on_script ###
return (df_usina,df_ug)
"""
### end_function ###

' ### include_on_script ###\nreturn (df_usina,df_ug)\n'

In [79]:
#### start_function ### function_name = export_biu_files
### start_function_vars ###
### start_funcion_defined_vars ###
### end_funcion_defined_vars ###
biu_download_path = os.path.join(biu_path,'Estagio_I')
casos = ['I','I_a','BIU']

""" ### include_on_script ###
df_usina = df_usina
df_ug = df_ug
"""
### end_function_vars ###
cols_ug = ['IdeUsinaOutorga','NumUgUsina','MdaPotenciaUnitaria','PrevisaoOC_regra','Justificativadaprevisao_new','CriterioPrevisao','DscCriterioPrevisao','CalculoPrevisorOC','OC_Obrigacao','PrevisaoOC_rapeel_max','DscJustificativaPrevisaoAtual','DatMonitoramento','DthEnvio','FASE']

cols_revisar_IO = ['IdeUsinaOutorga','DatInicioObraOutorgado','prev_IO_rapeel','prev_IO_SFG','DatMonitoramento','DthEnvio']

if 'I' in casos:
    # Caso I
    file_name = os.path.join(biu_download_path,f"caso_I.xlsx")
    df_ug[df_ug.caso_I][cols_ug].to_excel(file_name,index=False)

# Caso II_a
if 'II_a' in casos:
    file_name = os.path.join(biu_download_path,f"caso_II_a.xlsx")
    df_usina[df_usina.caso_II_a][cols_revisar_IO].to_excel(file_name,index=False)

if 'BIU' in casos:
    # Caso II-b e III
    file_name = os.path.join(biu_download_path,f"BIU.xlsx")
    df_usina[df_usina.caso_II_b | df_usina.caso_III].to_excel(file_name,index=False)

    file_name = os.path.join(biu_download_path,f"Usinas_reazer_robot.xlsx")
    df_usina[df_usina.caso_II_a | df_usina.caso_II_b][['IdeUsinaOutorga']].to_excel(file_name,index=False)
### end_function ###

## Análise de casos BIU

In [ ]:
list_criterios_BIU = ['manual', 'revisar_IO','homologar_marcos', 'em_teste','sem_monitoramento']
casos = df_usina[list_criterios_BIU +  list_casos].value_counts().reset_index().sort_values(by='caso_III')
casos
mask_casos_selecionados = casos[list_casos].any(axis=1)
casos[mask_casos_selecionados]

In [ ]:
df_usina.loc[df_usina.selecionado_BIU,list_casos+ ['manual','homologar_marcos']].value_counts().reset_index()

In [ ]:
#df_usina[df_usina.homologar_marcos & (~ df_usina.sem_monitoramento)][['IdeUsinaOutorga','NomUsina','homologar_marcos',"sem_monitoramento",'DatMonitoramento','DthEnvio','dsc_marcos_a_homologar']].to_excel("Usinas_sem_fotos.xlsx",index=False)

In [ ]:
raise

# Testes

## Junta dados do SAS e Python

In [ ]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})[["IdeUsinaOutorga","NumOperacaoUg",'CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo','CalculoPrevisorOC_mx',"FASE"]]

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True


#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
#criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [ ]:
criterios_da_previsao_sas_gb = criterios_da_previsao_sas.groupby("IdeUsinaOutorga")[['CriterioNovoPMO','DscCriterioNovo','DscJustificativaRegranova']].nunique()
usinas_dois_criterios = criterios_da_previsao_sas_gb[criterios_da_previsao_sas_gb.CriterioNovoPMO>1].index

In [ ]:
criterios_da_previsao_sas_usina = criterios_da_previsao_sas[['IdeUsinaOutorga','CriterioNovoPMO','DscJustificativaRegranova','DscCriterioNovo']].drop_duplicates()

In [ ]:
merge_usina  = pd.merge(df_usina_criterio[['IdeUsinaOutorga','criterio_novo_pmo','dsc_justificativa_regra_nova','dsccriterionovo']],
criterios_da_previsao_sas_usina,on=['IdeUsinaOutorga'],suffixes=('_py','_sas'),how='outer')

In [ ]:
criterios_da_previsao_sas_ug = criterios_da_previsao_sas[list_id_ug + ['CalculoPrevisorOC_mx','FASE']]

In [ ]:
merge_ug = pd.merge(df_ug[list_id_ug + ['FASE','Previsao_OC']],criterios_da_previsao_sas_ug,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge_ug['OC_Diff'] = merge_ug.Previsao_OC - merge_ug.CalculoPrevisorOC_mx

## Teste previsão

In [ ]:
merge_ug.sort_values(by='OC_Diff',ascending=False).head(20)

## Teste critérios

### FASE ok

In [ ]:
merge_ug[merge_ug.FASE_py != merge_ug.FASE_sas]

### criterio_novo_pmo OK

In [ ]:
merge_usina[(merge_usina.CriterioNovoPMO != merge_usina.criterio_novo_pmo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]

In [ ]:
merge_usina[(merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))].sample(6)

### dsc_justificativa_regra_nova OK

In [ ]:
merge_usina[(merge_usina.dsc_justificativa_regra_nova != merge_usina.DscJustificativaRegranova) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]


In [ ]:
merge_usina[(merge_usina.dsc_justificativa_regra_nova != merge_usina.DscJustificativaRegranova)]

### dsccriterionovo OK

In [ ]:
merge_usina[(merge_usina.dsccriterionovo != merge_usina.DscCriterioNovo) & (~merge_usina.IdeUsinaOutorga.isin(usinas_dois_criterios))]